In [ ]:
import os
import zipfile
import shutil
import glob
import netCDF4 as nc
from netCDF4 import Dataset
import rasterio
import rasterio.mask
from rasterio.transform import from_origin
import numpy as np
from osgeo import gdal, osr
import xarray as xr
import rioxarray as rio
from rioxarray.exceptions import NoDataInBounds
import matplotlib
from matplotlib.colors import ListedColormap, BoundaryNorm, Normalize
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import pandas as pd
import pyproj
import shapefile
from shapely.geometry import shape
from shapely.geometry import mapping
import rasterio.mask
import fiona
from scipy.interpolate import griddata
from descartes import PolygonPatch

In [2]:
# Set the base directory
base_directory = "/Users/lopezama/Documents/Blackwood/MERIS/download_test/data/20120402T180502_use"

In [ ]:
############### Step 1 #########################

# Assigns lat/lon to TSM_NN.nc from geo_coordinates.nc

# Load NetCDFs
tsm_nc = os.path.join(base_directory, "tsm_nn.nc")
geocoord_nc = os.path.join(base_directory, "geo_coordinates.nc")

if not os.path.exists(tsm_nc) or not os.path.exists(geocoord_nc):
    raise FileNotFoundError("One or both NetCDF files are missing in the directory.")

ds_tsm = xr.open_dataset(tsm_nc)
ds_geocoord = xr.open_dataset(geocoord_nc)

# Extract latitude & longitude
lat = ds_geocoord["latitude"].values  
lon = ds_geocoord["longitude"].values  

# Extract TSM data and ensure correct shape
var_name = list(ds_tsm.data_vars.keys())[0]  # Get the first variable name
data_values = ds_tsm[var_name].values  

# If data is 3D (e.g., time, lat, lon), select the first time step
if data_values.ndim == 3:
    data_values = data_values[0, :, :]

# Expected shape from TSM data
expected_shape = data_values.shape  

# Debugging: Print original shapes
print(f"🔹 Original Shapes:")
print(f"  - Latitude: {lat.shape}")
print(f"  - Longitude: {lon.shape}")
print(f"  - Data ({var_name}): {data_values.shape}")

# If lat/lon are 1D, convert to 2D
if lat.ndim == 1 and lon.ndim == 1:
    lon, lat = np.meshgrid(lon, lat)

# Debugging: Print shapes after conversion
print(f"🔹 Shapes After Conversion (if applied):")
print(f"  - Latitude: {lat.shape}")
print(f"  - Longitude: {lon.shape}")
print(f"  - Data ({var_name}): {data_values.shape}")

# Ensure final shapes match
if lat.shape != expected_shape or lon.shape != expected_shape:
    raise ValueError(f"Lat/Lon shape mismatch: Expected {expected_shape}, Got {lat.shape} & {lon.shape}")

# Create new NetCDF dataset
ds_new = xr.Dataset(
    {
        var_name: (["y", "x"], data_values)
    },
    coords={
        "latitude": (["y", "x"], lat),
        "longitude": (["y", "x"], lon)
    }
)

# Save NetCDF
output_nc = os.path.join(base_directory, "tsm_nn_spdm_test.nc")
ds_new.to_netcdf(output_nc)

print(f"Successfully saved NetCDF: {output_nc}")



############### Step 2 #########################

# Confirm NetCDF file with lat/lon as dimensions
tsm_nc_spdm_view = xr.open_dataset("/Users/lopezama/Documents/Blackwood/MERIS/scripts/earth_data/test_data/20110731T182333_use/tsm_nn_spdm_test.nc")
tsm_nc_spdm_view



############### Step 3 #########################

# Show min/max values of a variable in a netCDF
# MODIFY BEFORE USE - see bottom of step 3

def find_min_max(file_path, variable_name):
    """
    Finds the minimum and maximum values of a specified variable within a NetCDF file.

    Args:
        file_path (str): The path to the NetCDF file.
        variable_name (str): The name of the variable to analyze.

    Returns:
        tuple: A tuple containing the minimum and maximum values.
    """
    try:
        with Dataset(file_path, 'r') as nc_file:
            if variable_name not in nc_file.variables:
                raise ValueError(f"Variable '{variable_name}' not found in the file.")
            
            variable_data = nc_file.variables[variable_name][:]
            
            if np.size(variable_data) == 0:
                 raise ValueError(f"Variable '{variable_name}' has no data.")

            min_value = np.nanmin(variable_data)
            max_value = np.nanmax(variable_data)
            
            if np.isnan(min_value) or np.isnan(max_value):
                raise ValueError(f"Could not determine min/max values for '{variable_name}'. Ensure the data does not exclusively contain NaN values.")

            return min_value, max_value

    except FileNotFoundError:
        raise FileNotFoundError(f"File not found: {file_path}")
    except Exception as e:
         raise Exception(f"An error occurred: {e}")

if __name__ == '__main__':
    file_path = '/Users/lopezama/Documents/Blackwood/MERIS/scripts/earth_data/test_data_old/20120402T180502_test/tsm_nn_spdm_test.nc'
    #variable_name = 'TSM_NN'
    variable_name = 'latitude'
    #variable_name = 'longitude'

    try:
        min_val, max_val = find_min_max(file_path, variable_name)
        print(f"Minimum value of '{variable_name}': {min_val}")
        print(f"Maximum value of '{variable_name}': {max_val}")
    except Exception as e:
        print(f"Error: {e}")




############### Step 4 #########################

# Show values of first 20 pixels of a variable in a netCDF
# MODIFY BEFORE USE - see bottom of step 4

def print_netcdf_to_dataframe(file_path, variable_name):
    """
    Prints data from a NetCDF file to a Pandas DataFrame for the first 10 pixels.

    Args:
        file_path (str): Path to the NetCDF file.
        variable_name (str): Name of the variable to extract.
    """
    try:
        with nc.Dataset(file_path, 'r') as nc_file:
            variable_data = nc_file.variables[variable_name][:]

            if variable_data.ndim < 2:
                print("Variable must have at least 2 dimensions (e.g., latitude, longitude).")
                return

            num_pixels = min(20, variable_data.shape[-1])
            data_values = variable_data[..., :num_pixels]

            df = pd.DataFrame(data=data_values.reshape(-1, num_pixels))
            print(df)

    except FileNotFoundError:
        print(f"Error: File not found: {file_path}")
    except KeyError:
        print(f"Error: Variable '{variable_name}' not found in the file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# Path to netcdf you want to look at and variable you want to look at
    file_path = '/Users/lopezama/Documents/Blackwood/MERIS/scripts/earth_data/test_data_old/20120402T180502_test/tsm_nn_spdm_test.nc'
    #variable_name = 'TSM_NN'
    variable_name = 'latitude'
    #variable_name = 'longitude'

# Display the values
print_netcdf_to_dataframe(file_path, variable_name)



############### Step 5 #########################
# Look at netCDF details (dimensions, coordinates, variables)

nc_file = "/Users/lopezama/Documents/Blackwood/MERIS/scripts/earth_data/test_data/20110731T182333_use/tsm_nn_spdm_test.nc"

ds = xr.open_dataset(nc_file)
print(ds)  # Print dataset structure

# Check the variable names
print("Variables:", list(ds.variables.keys()))

# Check latitude & longitude values
print("Latitude sample:", ds["latitude"].values[:10])
print("Longitude sample:", ds["longitude"].values[:10])




############### Step 6 #########################
# Plot netCDF before clipping with ROI

# Load NetCDF
nc_file = nc.Dataset('/Users/lopezama/Documents/Blackwood/MERIS/scripts/earth_data/test_data/20110731T182333_use/tsm_nn_spdm_test.nc', 'r')

lat = nc_file.variables['latitude'][:]
lon = nc_file.variables['longitude'][:]
data = nc_file.variables['TSM_NN'][:]

# Mask NaN values in data
data_masked = np.ma.masked_invalid(data)

# Setup Basemap
fig, ax = plt.subplots(figsize=(10, 8))
map = Basemap(projection='cyl', 
              llcrnrlat=np.min(lat), urcrnrlat=np.max(lat),
              llcrnrlon=np.min(lon), urcrnrlon=np.max(lon), 
              resolution='i', ax=ax)

# Use imshow for plotting data
img = map.imshow(data_masked, extent=[np.min(lon), np.max(lon), np.min(lat), np.max(lat)], 
                 origin="upper", cmap="jet", norm=Normalize(vmin=np.nanmin(data), vmax=np.nanmax(data)))

# Draw map details
map.drawcoastlines()
map.drawcountries()
map.drawstates()  # Add state boundaries

# Add latitude & longitude labels
parallels = np.arange(np.floor(np.min(lat)), np.ceil(np.max(lat)), 2)
meridians = np.arange(np.floor(np.min(lon)), np.ceil(np.max(lon)), 2)

map.drawparallels(parallels, labels=[True, False, False, False], linewidth=0.2, fontsize=10)
map.drawmeridians(meridians, labels=[False, False, False, True], linewidth=0.2, fontsize=10)

# Load and plot a shapefile
shapefile_path = "/Users/lopezama/Documents/Blackwood/MERIS/ROI/west_us_poly_ll/west_us_poly_ll"  # Exclude .shp extension
map.readshapefile(shapefile_path, name="region", linewidth=0.5, color="purple")

# Fill shapefile area with purple (adjust transparency)
sf = shapefile.Reader(shapefile_path)
for shape_rec in sf.shapeRecords():
    poly_shape = shape_rec.shape.__geo_interface__
    patch = PolygonPatch(poly_shape, fc="purple", ec="purple", alpha=0.2, linewidth=0.5)
    ax.add_patch(patch)

# Properly attach colorbar
cbar = plt.colorbar(img, ax=ax, orientation="vertical", label='TSM_NN [g.m-3]')

# Add title
plt.title('TSM NetCDF with ROI')

plt.show()

In [ ]:
# ERROR 1 NoDataInBounds occurs in step 6

############### Step 6 #########################
# Clip TSM netCDF using lat/lon ROI shapefile (runs but results in NoDataInBounds so no clip is done)

def crop_netcdf_with_shapefile(nc_file, shapefile, output_nc, data_var="TSM_NN"):
    """
    Crops a NetCDF file using a shapefile and logs a message if no data is found.

    :param nc_file: Path to the input NetCDF file.
    :param shapefile: Path to the shapefile defining the crop region.
    :param output_nc: Path to save the cropped NetCDF file.
    :param data_var: The variable to extract from the NetCDF.
    """
    # Load NetCDF
    ds = xr.open_dataset(nc_file)

    # Ensure the dataset is georeferenced
    if "spatial_ref" not in ds:
        ds = ds.rio.write_crs("EPSG:4326")  # Set default CRS if missing

    # Load the shapefile
    gdf = gpd.read_file(shapefile)

    # Convert to dataset CRS if different
    if gdf.crs is not None and gdf.crs != ds.rio.crs:
        gdf = gdf.to_crs(ds.rio.crs)

    # Get geometry from the shapefile
    geom = [mapping(geometry) for geometry in gdf.geometry]

    # Define log file path
    log_file = os.path.join(os.path.dirname(output_nc), "log.txt")

    try:
        # Attempt to clip the NetCDF using the shapefile geometry
        clipped_ds = ds.rio.clip(geom, gdf.crs, drop=True)

        # Check if there is data inside the clipped region
        if data_var in clipped_ds and clipped_ds[data_var].count() == 0:
            raise NoDataInBounds(f"No data found in bounds. Data variable: {data_var}")

        # Save the cropped NetCDF
        clipped_ds.to_netcdf(output_nc)
        print(f"✅ Cropped NetCDF saved to: {output_nc}")

    except NoDataInBounds as e:
        # Write log file if NoDataInBounds error occurs
        with open(log_file, "w") as log:
            log.write(f"NoDataInBounds: No data found in ROI bounds. Data variable: {data_var}.\n")
        print(f"⚠️ No data found in ROI bounds. Log saved: {log_file}")

# Example Usage
tsm_nc_spdm = "/Users/lopezama/Documents/Blackwood/MERIS/scripts/earth_data/test_data_old/20120401T184126_use/tsm_nn_spdm_test.nc"
shp_ll = "/Users/lopezama/Documents/Blackwood/MERIS/ROI/west_us_poly_ll/west_us_poly_ll.shp"
tsm_nc_spdm_clip = "/Users/lopezama/Documents/Blackwood/MERIS/scripts/earth_data/test_data_old/20120401T184126_use/tsm_nn_spdm_clip.nc"

crop_netcdf_with_shapefile(tsm_nc_spdm, shp_ll, tsm_nc_spdm_clip)

In [ ]:
# ERROR 2 incorrect georeference when converting to geotiff in step 7

############### Step 7 #########################
# Convert unclipped netCDF to geotiff (runs but geotiff looks wonky)

def netcdf_to_geotiff(nc_file, var_name, output_tif, resolution=0.01):
    """
    Converts a scattered lat/lon NetCDF variable to a GeoTIFF.

    :param nc_file: Path to the input NetCDF file.
    :param var_name: Variable name in the NetCDF to extract.
    :param output_tif: Path to save the output GeoTIFF.
    :param resolution: Desired resolution (degree step size for output grid).
    """
    print(f"🔹 Loading NetCDF file: {nc_file}")

    # Open NetCDF file
    ds = xr.open_dataset(nc_file)

    # Identify latitude and longitude variable names
    lat_name = "latitude" if "latitude" in ds else "lat"
    lon_name = "longitude" if "longitude" in ds else "lon"

    if lat_name not in ds or lon_name not in ds:
        raise ValueError(f"❌ Latitude ({lat_name}) or longitude ({lon_name}) not found in dataset.")

    lats = ds[lat_name].values
    lons = ds[lon_name].values
    data = ds[var_name].values

    print(f"🔍 Original Shapes - Lats: {lats.shape}, Lons: {lons.shape}, Data: {data.shape}")

    # Check if lat/lon are 2D or 1D
    if lats.ndim == 2 and lons.ndim == 2:
        print("🔄 Detected 2D latitude/longitude arrays. Flattening...")
        lats = lats.flatten()
        lons = lons.flatten()
        data = data.flatten()
    elif lats.ndim == 1 and lons.ndim == 1:
        print("🔄 Detected 1D latitude/longitude arrays. Creating meshgrid...")
        lon_grid, lat_grid = np.meshgrid(lons, lats)
        lats = lat_grid.flatten()
        lons = lon_grid.flatten()
        data = data.flatten()
    else:
        raise ValueError("❌ Latitude and longitude dimensions are unexpected.")

    print(f"🔍 Flattened Shapes - Lats: {lats.shape}, Lons: {lons.shape}, Data: {data.shape}")

    # Print number of NaN values before filtering
    print(f"❌ NaN Count - Lats: {np.isnan(lats).sum()}, Lons: {np.isnan(lons).sum()}, Data: {np.isnan(data).sum()}")

    # Remove NaN values
    valid_mask = ~np.isnan(lats) & ~np.isnan(lons) & ~np.isnan(data)
    lats, lons, data = lats[valid_mask], lons[valid_mask], data[valid_mask]

    print(f"✅ Valid points after filtering: {len(lats)}")

    if len(lats) == 0:
        raise ValueError("⚠️ No valid lat/lon/data points found after removing NaNs.")

    # Define output grid
    lat_min, lat_max = np.min(lats), np.max(lats)
    lon_min, lon_max = np.min(lons), np.max(lons)
    grid_lats = np.arange(lat_min, lat_max, resolution)
    grid_lons = np.arange(lon_min, lon_max, resolution)
    
    lon_grid, lat_grid = np.meshgrid(grid_lons, grid_lats)

    # Interpolate scattered data onto a regular grid
    print("🔄 Performing interpolation...")
    grid_data = griddata((lons, lats), data, (lon_grid, lat_grid), method='linear')

    # Define raster transform
    transform = from_origin(lon_min, lat_max, resolution, resolution)

    # Save to GeoTIFF
    print(f"💾 Saving GeoTIFF to {output_tif}")
    with rasterio.open(
        output_tif, 'w', 
        driver='GTiff', 
        height=grid_data.shape[0], width=grid_data.shape[1],
        count=1, dtype=grid_data.dtype,
        crs="EPSG:4326", transform=transform,
        nodata=np.nan
    ) as dst:
        dst.write(grid_data, 1)

    print("✅ GeoTIFF successfully created!")

# Example Usage
nc_file = "/Users/lopezama/Documents/Blackwood/MERIS/scripts/earth_data/test_data/20110731T182333_use/tsm_nn_spdm_test.nc"
var_name = "TSM_NN"  
output_tif = "/Users/lopezama/Documents/Blackwood/MERIS/scripts/earth_data/test_data/20110731T182333_use/tsm_nn_spdm_test.tif"

netcdf_to_geotiff(nc_file, var_name, output_tif, resolution=0.01)

# Plot unclipped geotiff (interpolation is wonky)

def view_geotiff(file_path):
    """
    Opens and displays a GeoTIFF file, handling NaN/Inf values properly.

    :param file_path: Path to the GeoTIFF file
    """
    with rasterio.open(file_path) as src:
        data = src.read(1).astype(np.float32)  # Read the first band and ensure float32

        # Handle NoData values
        if src.nodata is not None:
            data[data == src.nodata] = np.nan  # Convert NoData to NaN

        # Check if all values are NaN
        if np.all(np.isnan(data)):
            raise ValueError(f"Error: The dataset {file_path} contains only NaN values and cannot be plotted.")

        # Ensure valid extent
        extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]
        
        # Check if extent contains NaN or Inf
        if any(np.isnan(extent)) or any(np.isinf(extent)):
            raise ValueError(f"Error: The spatial extent contains NaN/Inf values: {extent}")

        # Mask invalid values for plotting
        data = np.ma.masked_invalid(data)

        # Plot the data
        plt.figure(figsize=(10, 6))
        plt.imshow(data, cmap="viridis", extent=extent, origin="upper")
        plt.colorbar(label="TSM_NN [g.m-3]")
        plt.title(f"TSM GeoTIFF")
        #plt.title(f"TSM GeoTIFF: {file_path}")
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.show()

# Example Usage
if __name__ == "__main__":
    tsm_tif = '/Users/lopezama/Documents/Blackwood/MERIS/scripts/earth_data/test_data/20110731T182333_use/tsm_nn_spdm_test.tif'  
    view_geotiff(tsm_tif)